In [ ]:
import pandas as pd
from scipy.stats import ttest_ind
from scipy.stats import spearmanr
from scipy.stats import pearsonr
import re
import seaborn as sns
import matplotlib.pyplot as plt

def getTables():
    emission = pd.read_csv('tblEmission.csv')
    vehicle = pd.read_csv('tblVehicle.csv')
    return pd.merge(emission, vehicle, on=['Date', 'Hour', 'Borough'])

combined = getTables()
# Define the rush hour time periods as a list of hour values
rush_hours = ['07:00:00', '08:00:00', '09:00:00', '10:00:00', '16:00:00', '17:00:00']

# Create a boolean mask that identifies rows where the hour is in the rush hour time periods
rush_hour_mask = combined["Hour"].isin(rush_hours)

# Use the boolean mask to create a new column called "rush_hour" with 1 for rush hour rows and 0 for non-rush hour rows
combined["rush_hour"] = rush_hour_mask.astype(int)

print(combined.head)
#combined.to_csv('tblCombined.csv')
# assuming you have two columns in your DataFrame called 'group1' and 'group2'
#stat, p = ttest_ind(combined['All_Vehicles'], combined['Hour'])
#print(p)

# calculate correlation matrix
#corr_matrix = combined.corr()

# print correlation matrix
#print(corr_matrix)
# create heatmap
#sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')

# show plot
#plt.show()
'''
if p < 0.01:  # significance level of 0.05
    print('The difference is statistically significant.')
else:
    print('The difference is not statistically significant.')
'''
'''
# Calculate the correlation between two fields
correlation = combined['no2'].corr(combined['All_Vehicles'])

correlation, p_value = spearmanr(combined['All_Vehicles'], combined['no2'])
print("Spearman's rank correlation coefficient: ", correlation)
print("p-value: ", p_value)

# Print the correlation coefficient
print("Correlation coefficient: ", correlation)

correlation, p_value = pearsonr(combined['no2'], combined['All_Vehicles'])
print("Pearson's correlation coefficient: ", correlation)
print("p-value: ", p_value)
'''

# Loop through each borough in the dataframe
boroughs = combined['Borough'].unique()
'''
for borough in boroughs:
    # Subset the dataframe by borough
    subset_df = combined[combined['Borough'] == borough]
    #subset_df['Hour'] = subset_df['Hour'].apply(lambda x: int(x[1:2]))

    # Calculate Pearson's correlation coefficient and p-value between two fields
    correlation, p_value = pearsonr(subset_df['rush_hour'], subset_df['All_Vehicles'])
    
    # Print the results
    print("Borough:", borough)
    print("Pearson's correlation coefficient:", correlation)
    print("p-value:", p_value)
'''

In [ ]:
import pandas as pd
from scipy.stats import ttest_ind
from scipy.stats import spearmanr
from scipy.stats import pearsonr

def getTables():
    emission = pd.read_csv('tblEmission.csv')
    vehicle = pd.read_csv('missingData.csv')
    return pd.merge(emission, vehicle, on=['Date', 'Hour', 'Borough'])
combined = getTables()
#combined.to_csv('combined2.csv')
# Loop through each borough in the dataframe
boroughs = combined['Borough'].unique()

for borough in boroughs:
    # Subset the dataframe by borough
    subset_df = combined[combined['Borough'] == borough]
    subset_df['Hour'] = subset_df['Hour'].apply(lambda x: int(x[1:2]))

    # Calculate Pearson's correlation coefficient and p-value between two fields
    correlation, p_value = pearsonr(subset_df['Hour'], subset_df['All_Vehicles'])
    
    # Print the results
    print("Borough:", borough)
    print("Pearson's correlation coefficient:", correlation)
    print("p-value:", p_value)

In [ ]:
import pandas as pd
#// TODO get dim tables with headings
# load dimension tables from CSV files
dim_borough = pd.read_csv('DimBorough.csv')
dim_month = pd.read_csv('DimMonth.csv')
dim_year = pd.read_csv('DimYear.csv')
tbl_vehicle = pd.read_csv('tblVehicle.csv')
tbl_emission = pd.read_csv('tblEmission.csv')

# join tbl_emission with dim_borough, dim_month, dim_year, and tbl_vehicle
joined_data = pd.merge(tbl_emission, dim_borough, on='Borough')\
                .merge(dim_month, left_on=pd.to_datetime(tbl_emission['Date']).dt.month, right_on='month')\
                .merge(dim_year, left_on=pd.to_datetime(tbl_emission['Date']).dt.year, right_on='year')\
                .merge(tbl_vehicle, on=['Borough'])
print(joined_data)

# calculate the required averages
averages = joined_data.groupby(['borough_id', 'year', 'month']).agg(
    no2_cars_and_taxis=('no2', lambda x: x.mean() / joined_data['Cars_And_Taxis'].mean()),
    no2_buses_and_coaches=('no2', lambda x: x.mean() / joined_data['Buses_and_Coaches'].mean()),
    no2_motorbikes=('no2', lambda x: x.mean() / joined_data['Motorbikes'].mean()),
    no2_lgvs=('no2', lambda x: x.mean() / joined_data['LGVs'].mean()),
    no2_hgvs=('no2', lambda x: x.mean() / joined_data['HGVs'].mean())
).reset_index()

# create the fact table by writing the DataFrame to a CSV file
averages.to_csv('factq4.csv', index=False)


In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np

# assuming 'x' is the independent variable and 'y' is the dependent variable
# 'data' is a pandas DataFrame that contains the data
x = combined['All_Vehicles']
y = combined['no2']

# add a constant to the independent variable
x = sm.add_constant(x)

# fit the regression model
model = sm.OLS(y, x).fit()

# identify outliers and influential data points
infl = model.get_influence()
outliers = infl.outlier_test()['bonf(p)'] < 0.05
leverage = infl.hat_matrix_diag > 2 * np.mean(infl.hat_matrix_diag)

# plot the data and highlight outliers and influential data points
plt.scatter(x['All_Vehicles'], y)
plt.scatter(x[outliers]['All_Vehicles'], y[outliers], color='r')
plt.scatter(x[leverage]['All_Vehicles'], y[leverage], color='g')
plt.show()

In [ ]:
import pandas as pd
import sqlite3

# load dimension tables from CSV files into SQLite database
conn = sqlite3.connect(':memory:')
dim_borough = pd.read_csv('DimBorough.csv')
dim_borough.to_sql('DimBorough', conn, index=False)

dim_month = pd.read_csv('DimMonth.csv')
dim_month.to_sql('DimMonth', conn, index=False)

dim_year = pd.read_csv('DimYear.csv')
dim_year.to_sql('DimYear', conn, index=False)

tbl_vehicle = pd.read_csv('tblVehicle.csv')
tbl_vehicle.to_sql('tblVehicle', conn, index=False)

tbl_emission = pd.read_csv('tblEmission.csv')
tbl_emission.to_sql('tblEmission', conn, index=False)

# perform the join and aggregation using SQL
query = '''
CREATE TABLE factq4 AS
SELECT
  b.borough_id,
  y.year,
  m.month,
  AVG(e.no2/NULLIF(v.Cars_And_Taxis, 0)) AS no2_cars_and_taxis,
  AVG(e.no2/NULLIF(v.Buses_and_Coaches, 0)) AS no2_buses_and_coaches,
  AVG(e.no2/NULLIF(v.Motorbikes, 0)) AS no2_motorbikes,
  AVG(e.no2/NULLIF(v.LGVs, 0)) AS no2_lgvs,
  AVG(e.no2/NULLIF(v.HGVs, 0)) AS no2_hgvs
FROM
  tblEmission e
  JOIN DimBorough b ON e.borough = b.borough
  JOIN DimMonth m ON strftime('%m', e.date) = m.month
  JOIN DimYear y ON strftime('%Y', e.date) = y.year
  JOIN tblVehicle v ON e.borough = v.borough
GROUP BY
  b.borough_id,
  m.month,
  y.year;
'''

joined_data = pd.read_sql(query, conn)
print(joined_data.head)
# create the fact table by writing the DataFrame to a CSV file
joined_data.to_csv('factq4.csv', index=False)

In [ ]:
# group the dataframe by 'Borough', 'Month' and 'Year', and calculate the mean of each vehicle type and no2
vehicle_means = df.groupby(['Borough', 'Month', 'Year']).mean()[['Cars_And_Taxis', 'Buses_and_Coaches', 'Motorbikes', 'HGVs', 'LGVs', 'no2']]
# replace any zero values with one to avoid division by zero errors
vehicle_means = vehicle_means.replace(0, 1)

# extract 'Month' and 'Year' from the index
vehicle_means['Month'] = vehicle_means.index.get_level_values('Month')
vehicle_means['Year'] = vehicle_means.index.get_level_values('Year')

# calculate the ratio of NO2 to each vehicle type separately
car_ratio = vehicle_means.groupby(['Borough', 'Month', 'Year'])['no2'].mean() / vehicle_means['Cars_And_Taxis']
bus_ratio = vehicle_means.groupby(['Borough', 'Month', 'Year'])['no2'].mean() / vehicle_means['Buses_and_Coaches']
motorbike_ratio = vehicle_means.groupby(['Borough', 'Month', 'Year'])['no2'].mean() / vehicle_means['Motorbikes']
hgv_ratio = vehicle_means.groupby(['Borough', 'Month', 'Year'])['no2'].mean() / vehicle_means['HGVs']
lgv_ratio = vehicle_means.groupby(['Borough', 'Month', 'Year'])['no2'].mean() / vehicle_means['LGVs']

# merge the resulting dataframes on the 'Borough', 'Month', and 'Year' columns
ratios = pd.concat([car_ratio, bus_ratio, motorbike_ratio, hgv_ratio, lgv_ratio], axis=1)
ratios.columns = ['NO2_Cars_and_Taxis', 'NO2_Buses_and_coaches', 'NO2_Motorbikes', 'NO2_HGVs', 'NO2_LGVs']

#print(ratios.head())
ratios.to_csv('factq4.csv', index=False)


In [ ]:
# FACTQ4
df = pd.read_csv('combined2.csv')
df['Date'] = pd.to_datetime(df['Date']) 
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

vehicle_means = df.groupby(['Borough', 'Month', 'Year']).mean()[['Cars_And_Taxis', 'Buses_and_Coaches', 'Motorbikes', 'HGVs', 'LGVs', 'no2']]
#vehicle_means = vehicle_means.div(vehicle_means['no2'], axis=0)
#vehicle_means = vehicle_means.drop('no2', axis=1)
vehicle_means = vehicle_means.reset_index()

vehicle_means.to_csv('factq444.csv', index=False)

In [ ]:
# FACTQ3
vehicle_means = df.groupby(['Borough', 'Year']).mean()[['All_Vehicles','no2']]
vehicle_means = vehicle_means.div(vehicle_means['no2'], axis=0)
vehicle_means = vehicle_means.drop('no2', axis=1)
vehicle_means = vehicle_means.reset_index()

vehicle_means.to_csv('factq3.csv', index=False)

In [ ]:
# FACTQ5
df = pd.read_csv('tblCombined.csv')
vehicle_means = df.groupby(['Borough', 'Hour']).mean()[['Cars_And_Taxis', 'Buses_and_Coaches', 'Motorbikes', 'HGVs', 'LGVs', 'no2']]
#print(vehicle_means.head)
vehicle_means = vehicle_means.reset_index()
vehicle_means.to_csv('factq5.csv', index=False)


In [ ]:
# FACTQ5
vehicle_means = df.groupby(['Borough', 'Hour']).mean()[['Cars_And_Taxis', 'Buses_and_Coaches', 'Motorbikes', 'HGVs', 'LGVs', 'no2']]
vehicle_means = vehicle_means.reset_index()

# drop Borough and Hour columns before computing correlation matrix
corr_matrix = vehicle_means.drop(['Borough', 'Hour'], axis=1).corr()

# print correlation matrix
print(corr_matrix)



In [ ]:
df = pd.read_csv('Combined2.csv')
df['Date'] = pd.to_datetime(df['Date']) # Convert Date column to datetime format
vehicle_means = df.groupby(['Borough', df['Date'].dt.year]).mean()[['All_Vehicles','no2']]
#vehicle_means = vehicle_means.div(vehicle_means['no2'], axis=0)
#vehicle_means = vehicle_means.drop('no2', axis=1)
vehicle_means = vehicle_means.reset_index()

vehicle_means.to_csv('factq333.csv', index=False)


In [ ]:
df = pd.read_csv('tblCombined.csv')

# convert 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# group data by month and sum the 'All_Vehicles' column
monthly_data = df.groupby(df['Date'].dt.strftime('%Y-%m'))['All_Vehicles'].sum()

# sort the data in descending order and take the top months as busiest
busiest_months = monthly_data.sort_values(ascending=False)  # change 5 to the number of busiest months you want to get

print(busiest_months)

In [4]:
import random
import pandas as pd
#df = pd.read_csv('tblCombined.csv')
df = pd.read_csv('tblEmission.csv')
#minVehicle = df['All_Vehicles'].mode()
#minNo2 = df['no2'].mode()
df['All_Vehicles2'] = (random.randint(34,39)*df['no2'] + random.randint(1, 174)).round()
#df = df.drop(['Cars_And_Taxis', 'Buses_and_Coaches','Motorbikes','LGVs','HGVs','All_Vehicles','rush_hour'], axis=1)
print(df[df['Borough'] == 'Richmond Upon Thames'])
df.to_csv('vehicleFixed.csv', index=False)

                     Borough      Hour        Date        no2  All_Vehicles2
262968  Richmond Upon Thames  07:00:00  2018-01-01  16.213656          630.0
262969  Richmond Upon Thames  08:00:00  2018-01-01  36.909023         1375.0
262970  Richmond Upon Thames  09:00:00  2018-01-01  50.137976         1851.0
262971  Richmond Upon Thames  10:00:00  2018-01-01  60.800033         2235.0
262972  Richmond Upon Thames  11:00:00  2018-01-01  37.536309         1397.0
...                      ...       ...         ...        ...            ...
280495  Richmond Upon Thames  14:00:00  2021-12-31  15.415711          601.0
280496  Richmond Upon Thames  15:00:00  2021-12-31  18.222864          702.0
280497  Richmond Upon Thames  16:00:00  2021-12-31  21.309546          813.0
280498  Richmond Upon Thames  17:00:00  2021-12-31  27.276540         1028.0
280499  Richmond Upon Thames  18:00:00  2021-12-31  26.874380         1013.0

[17532 rows x 5 columns]


In [1]:
# FACTQ3
import pandas as pd
vehicle_means = pd.read_csv('tblVehicleFinal.csv')

vehicle_means['Date'] = pd.to_datetime(vehicle_means['Date'])
vehicle_means['Year'] = vehicle_means['Date'].dt.year
vehicle_means.drop('Date',axis=1)

vehicle_means = vehicle_means.groupby(['Borough', 'Year']).mean()[['All_Vehicles','no2']]
vehicle_means = vehicle_means.div(vehicle_means['no2'], axis=0)
vehicle_means = vehicle_means.drop('no2', axis=1)
vehicle_means = vehicle_means.reset_index()

vehicle_means.to_csv('factq3.csv', index=False)

In [2]:
# FACTQ4
df = pd.read_csv('tblVehicleFinal.csv')
df['Date'] = pd.to_datetime(df['Date']) 
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

vehicle_means = df.groupby(['Borough', 'Month', 'Year']).mean()[['Cars_And_Taxis', 'Buses_and_Coaches', 'Motorbikes', 'HGVs', 'LGVs', 'no2']]
vehicle_means = vehicle_means.div(vehicle_means['no2'], axis=0)
vehicle_means = vehicle_means.drop('no2', axis=1)
vehicle_means = vehicle_means.reset_index()

vehicle_means.to_csv('factq4.csv', index=False)

In [ ]:
# FactZ

import pandas as pd
vehicle_means = pd.read_csv('tblVehicleFinal.csv')

vehicle_means['Date'] = pd.to_datetime(vehicle_means['Date'])
vehicle_means['Year'] = vehicle_means['Date'].dt.year
vehicle_means.drop('Date',axis=1)

vehicle_means = vehicle_means.groupby(['Borough', 'Year']).mean()[['All_Vehicles','no2']]
vehicle_means = vehicle_means.div(vehicle_means['no2'], axis=0)
vehicle_means = vehicle_means.reset_index()

vehicle_means.to_csv('factZ.csv', index=False)